In [2]:
# A1 - Configuração base (uso LOCAL, sem Google Drive)

from pathlib import Path
import json
import numpy as np
import pandas as pd

# Pasta base ONDE vão ficar todos os CSV/JSON
# Agora usando caminho RELATIVO ao projeto / notebook:
# - Se existir uma pasta "airnet_inputs" na pasta atual, usa ela.
# - Senão, procura "airnet_inputs" na pasta mãe (por ex. quando o notebook está em "notebooks/").
NOTEBOOK_DIR = Path().resolve()
if (NOTEBOOK_DIR / "airnet_inputs").exists():
    BASE_DIR = (NOTEBOOK_DIR / "airnet_inputs").resolve()
else:
    BASE_DIR = (NOTEBOOK_DIR.parent / "airnet_inputs").resolve()

BASE_DIR.mkdir(parents=True, exist_ok=True)

print("Arquivos vão ser salvos em:", BASE_DIR)

# ===========================
# CONFIGURAÇÃO GLOBAL
# ===========================

DAY_START = 6 * 60    # 06:00 em minutos
DAY_END   = 22 * 60   # 22:00 em minutos
SLOT_MIN  = 60        # todos os voos em horas inteiras

TURNAROUND_MIN = 60   # mínimo entre pouso e nova decolagem (aeronave)
MIN_CONN_MIN   = 60   # conexão mínima passageiro

FLEET_N   = 250       # aeronaves
SEATS_PER = 200       # assentos por aeronave

print("Config:")
print(" - Dia:", DAY_START, "->", DAY_END, "(min)")
print(" - Slot:", SLOT_MIN, "min")
print(" - Frota:", FLEET_N, "aeronaves de", SEATS_PER, "lugares")


Arquivos vão ser salvos em: C:\Users\lucas\Projects\Softwares\PSI3472\airnet_inputs
Config:
 - Dia: 360 -> 1320 (min)
 - Slot: 60 min
 - Frota: 250 aeronaves de 200 lugares


In [3]:
# A2 - Definir os 14 aeroportos (códigos do enunciado) e salvar

# Ordem PADRÃO em todas as tabelas
airport_codes = [
    "MA", "BE", "FO", "NA", "RE", "SA", "BR",
    "BH", "CI", "RJ", "SP", "CR", "FL", "PA"
]

airport_names = [
    "Manaus",
    "Belém",
    "Fortaleza",
    "Natal",
    "Recife",
    "Salvador",
    "Brasília",
    "Belo Horizonte",
    "Cuiabá",
    "Rio de Janeiro",
    "São Paulo",
    "Curitiba",
    "Florianópolis",
    "Porto Alegre",
]

# lat/lon aproximados só pra plot / mapa (não importam para o GA)
airport_lat = [
    -3.1190,   # MA - Manaus
    -1.4558,   # BE - Belém
    -3.7172,   # FO - Fortaleza
    -5.7945,   # NA - Natal
    -8.0476,   # RE - Recife
    -12.9777,  # SA - Salvador
    -15.7939,  # BR - Brasília
    -19.9167,  # BH - Belo Horizonte
    -15.6010,  # CI - Cuiabá
    -22.9068,  # RJ - Rio de Janeiro
    -23.5505,  # SP - São Paulo
    -25.4284,  # CR - Curitiba
    -27.5954,  # FL - Florianópolis
    -30.0346,  # PA - Porto Alegre
]

airport_lon = [
    -60.0217,  # MA
    -48.4902,  # BE
    -38.5434,  # FO
    -35.2110,  # NA
    -34.8770,  # RE
    -38.5016,  # SA
    -47.8828,  # BR
    -43.9345,  # BH
    -56.0979,  # CI
    -43.1729,  # RJ
    -46.6333,  # SP
    -49.2733,  # CR
    -48.5480,  # FL
    -51.2177,  # PA
]

airports_df = pd.DataFrame({
    "id": np.arange(len(airport_codes), dtype=np.int32),
    "code": airport_codes,
    "name": airport_names,
    "lat": airport_lat,
    "lon": airport_lon,
})

airports_csv_path  = BASE_DIR / "airports.csv"
airports_json_path = BASE_DIR / "airports.json"

airports_df.to_csv(airports_csv_path, index=False)

with open(airports_json_path, "w", encoding="utf-8") as f:
    json.dump({
        "airports": airports_df.to_dict(orient="records")
    }, f, indent=2, ensure_ascii=False)

print("Salvos:")
print(" -", airports_csv_path)
print(" -", airports_json_path)

# Oferecer download local


Salvos:
 - C:\Users\lucas\Projects\Softwares\PSI3472\airnet_inputs\airports.csv
 - C:\Users\lucas\Projects\Softwares\PSI3472\airnet_inputs\airports.json


In [4]:
# A3 - TAB_TV (tempos de voo) e rotas

codes = airport_codes
n = len(codes)

# TABELA DO ENUNCIADO (13 valores por linha, sem diagonal)
tv_rows = {
    "MA": [120, 180, 180, 180, 180, 180, 240, 180, 300, 300, 300, 360, 360],
    "BE": [120, 120, 120, 120, 180, 180, 240, 180, 240, 240, 300, 300, 360],
    "FO": [180, 120,  60,  60, 120, 180, 180, 240, 240, 240, 300, 300, 360],
    "NA": [180, 120,  60,  60, 120, 180, 180, 240, 240, 240, 300, 300, 360],
    "RE": [180, 120,  60,  60, 120, 180, 180, 240, 180, 180, 240, 240, 300],
    "SA": [180, 180, 120, 120, 120, 120, 120, 180, 120, 120, 180, 180, 240],
    "BR": [180, 180, 180, 180, 180, 120,  60,  60, 120, 120, 180, 180, 240],
    "BH": [240, 240, 180, 180, 180, 120,  60, 120,  60,  60, 120, 120, 180],
    "CI": [180, 180, 240, 240, 240, 180,  60, 120, 180, 180, 180, 180, 240],
    "RJ": [300, 240, 240, 240, 180, 120, 120,  60, 180,  60, 120, 120, 180],
    "SP": [300, 240, 240, 240, 180, 120, 120,  60, 180,  60,  60,  60, 120],
    "CR": [300, 300, 300, 300, 240, 180, 180, 120, 180, 120,  60,  60,  60],
    "FL": [360, 300, 300, 300, 240, 180, 180, 120, 180, 120,  60,  60,  60],
    "PA": [360, 360, 360, 360, 300, 240, 240, 180, 240, 180, 120,  60,  60],
}

# Montar matriz 14x14 com diag=0
TAB_TV = np.zeros((n, n), dtype=int)

for i, orig in enumerate(codes):
    vals = tv_rows[orig]
    k = 0
    for j, dest in enumerate(codes):
        if i == j:
            TAB_TV[i, j] = 0
        else:
            TAB_TV[i, j] = vals[k]
            k += 1

# Checar (opcional): simetria aproximada
print("Matriz TAB_TV construída. Simétrica?:", np.all(TAB_TV == TAB_TV.T))

# Salvar TAB_TV em formato de matriz tipo slide do professor
tv_df = pd.DataFrame(TAB_TV, index=codes, columns=codes)
tv_csv_path = BASE_DIR / "tab_tv.csv"
tv_df.to_csv(tv_csv_path)
print("Salvo:", tv_csv_path)

# Construir lista de rotas (orig_id, dest_id, time_min) para JSON/CSV
routes = []
route_id = 0
for i, orig in enumerate(codes):
    for j, dest in enumerate(codes):
        if i == j:
            continue
        t = int(TAB_TV[i, j])
        if t <= 0:
            continue
        routes.append({
            "route_id": route_id,
            "orig_id": i,
            "dest_id": j,
            "orig_code": orig,
            "dest_code": dest,
            "time_min": t
        })
        route_id += 1

routes_df = pd.DataFrame(routes)

routes_csv_path  = BASE_DIR / "routes.csv"
routes_json_path = BASE_DIR / "routes.json"

routes_df.to_csv(routes_csv_path, index=False)
with open(routes_json_path, "w", encoding="utf-8") as f:
    json.dump({"routes": routes}, f, indent=2, ensure_ascii=False)

print("Salvos:")
print(" -", routes_csv_path)
print(" -", routes_json_path)
print("Total de rotas:", len(routes_df))



Matriz TAB_TV construída. Simétrica?: True
Salvo: C:\Users\lucas\Projects\Softwares\PSI3472\airnet_inputs\tab_tv.csv
Salvos:
 - C:\Users\lucas\Projects\Softwares\PSI3472\airnet_inputs\routes.csv
 - C:\Users\lucas\Projects\Softwares\PSI3472\airnet_inputs\routes.json
Total de rotas: 182


In [5]:
# A4 - TAB_OD (Origem-Destino, #passageiros) e passengers.json

od_rows = {
    "MA": [350, 200, 200, 300, 200, 500, 200, 200, 400, 800, 300, 200, 300],
    "BE": [400, 400, 300, 200, 300, 500, 200, 100, 400, 900, 300, 200, 200],
    "FO": [300, 350, 500, 400, 400, 600, 300, 200, 500, 900, 300, 300, 400],
    "NA": [200, 300, 400, 300, 400, 400, 200, 300, 400, 600, 300, 300, 300],
    "RE": [300, 400, 600, 400, 600, 500, 300, 300, 800, 800, 400, 400, 400],
    "SA": [200, 200, 800, 600, 600, 400, 400, 400, 800, 1100, 500, 500, 500],
    "BR": [500, 500, 900, 500, 700, 900, 700, 800, 1400, 2500, 600, 600, 800],
    "BH": [200, 300, 400, 200, 500, 200, 500, 200, 600, 900, 300, 200, 200],
    "CI": [200, 200, 300, 200, 200, 200, 300, 200, 200, 400, 100, 100, 200],
    "RJ": [600, 700, 900, 600, 1000, 900, 1600, 900, 900, 3400, 900, 800, 900],
    "SP": [800, 900, 1100, 700, 1400, 1300, 2500, 100, 800, 2800, 1500, 1200, 1100],
    "CR": [100, 200, 300, 200, 200, 400, 500, 300, 200, 600, 800, 300, 500],
    "FL": [100, 150, 200, 500, 300, 200, 600, 400, 200, 600, 600, 500, 300],
    "PA": [100, 250, 200, 400, 300, 400, 500, 300, 200, 700, 800, 300, 300],
}

TAB_OD = np.zeros((n, n), dtype=int)

for i, orig in enumerate(codes):
    vals = od_rows[orig]
    k = 0
    for j, dest in enumerate(codes):
        if i == j:
            TAB_OD[i, j] = 0
        else:
            TAB_OD[i, j] = vals[k]
            k += 1

od_df = pd.DataFrame(TAB_OD, index=codes, columns=codes)
od_csv_path = BASE_DIR / "tab_od.csv"
od_df.to_csv(od_csv_path)
print("Salvo:", od_csv_path)

total_demand = int(TAB_OD.sum())
print("Demanda total (TAB_OD):", total_demand)

# passengers.json em formato agregado (um registro por par OD)
passengers_agg = []
pid = 0
for i, orig in enumerate(codes):
    for j, dest in enumerate(codes):
        if i == j:
            continue
        d = int(TAB_OD[i, j])
        if d <= 0:
            continue
        passengers_agg.append({
            "id": pid,
            "orig_id": i,
            "dest_id": j,
            "orig_code": codes[i],
            "dest_code": codes[j],
            "demand": d
        })
        pid += 1

passengers_json_path = BASE_DIR / "passengers.json"
with open(passengers_json_path, "w", encoding="utf-8") as f:
    json.dump({"od_pairs": passengers_agg}, f, indent=2, ensure_ascii=False)

print("Salvo:", passengers_json_path)
print("N pares OD com demanda >", 0, ":", len(passengers_agg))



Salvo: C:\Users\lucas\Projects\Softwares\PSI3472\airnet_inputs\tab_od.csv
Demanda total (TAB_OD): 94800
Salvo: C:\Users\lucas\Projects\Softwares\PSI3472\airnet_inputs\passengers.json
N pares OD com demanda > 0 : 182


In [6]:
# A5 - Gerar fleet.json e meta.json (para o GA/Qt)

fleet = {
    "num_aircraft": int(FLEET_N),
    "seats_per_aircraft": int(SEATS_PER),
    "aircraft_ids": [f"AC_{i:03d}" for i in range(FLEET_N)],
}

fleet_path = BASE_DIR / "fleet.json"
with open(fleet_path, "w", encoding="utf-8") as f:
    json.dump(fleet, f, indent=2, ensure_ascii=False)
print("Salvo:", fleet_path)

meta = {
    "day_start": DAY_START,
    "day_end": DAY_END,
    "slot_min": SLOT_MIN,
    "turnaround_min": TURNAROUND_MIN,
    "min_conn_min": MIN_CONN_MIN,
    "fleet_n": FLEET_N,
    "seats_per_aircraft": SEATS_PER,
    "airport_order": codes,
    "total_demand_od": int(TAB_OD.sum()),
}

meta_path = BASE_DIR / "meta.json"
with open(meta_path, "w", encoding="utf-8") as f:
    json.dump(meta, f, indent=2, ensure_ascii=False)
print("Salvo:", meta_path)

print("\nResumo final dos arquivos na pasta:")
for p in BASE_DIR.iterdir():
    print(" -", p.name)


Salvo: C:\Users\lucas\Projects\Softwares\PSI3472\airnet_inputs\fleet.json
Salvo: C:\Users\lucas\Projects\Softwares\PSI3472\airnet_inputs\meta.json

Resumo final dos arquivos na pasta:
 - airports.csv
 - airports.json
 - Brazil_Blank_Map.svg
 - Brazil_states_blank.png
 - fleet.json
 - flights_ga.json
 - forbidden_routes.json
 - ga_stats.json
 - meta.json
 - passengers.json
 - passengers_flights.json
 - relatorio_trafego_aereo.pdf
 - routes.csv
 - routes.json
 - TAB_AP_passageiros_grupos.csv
 - TAB_AV_voos.csv
 - tab_od.csv
 - tab_tv.csv
